In this notebook we have used following resources:

https://www.tensorflow.org/guide/intro_to_graphs

https://jonathan-hui.medium.com/tensorflow-eager-execution-v-s-graph-tf-function-6edaa870b1f1

https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6



In [1]:
import tensorflow as tf

## Creating TensorFlow Graphs with `tf.function`



In [4]:
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
print("Eager function value:")
print(orig_value)

print("Graph mode function value:")
print(tf_function_value)
assert(orig_value == tf_function_value)

Eager function value:
[[12.]]
Graph mode function value:
[[12.]]


In [5]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: 1
Second branch, with graph: 0


In [6]:
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  

## Benchmarking two modes

In [ ]:
import timeit

In [ ]:
def eager_function(x):
  result = tf.matmul(x, x)
  return result

x = tf.constant([[1.0, 2.0], [3.0, 4.0]])
graph_function = tf.function(eager_function)
graph_function(x)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 7., 10.],
       [15., 22.]], dtype=float32)>

In [ ]:
print("Eager time:", timeit.timeit(lambda: eager_function(x), number=1))
print("Graph time:", timeit.timeit(lambda: graph_function(x), number=1))

Eager time: 0.0004805379999197612
Graph time: 0.002319958000043698


In [ ]:
print("Eager time:", timeit.timeit(lambda: eager_function(x), number=10000))
print("Graph time:", timeit.timeit(lambda: graph_function(x), number=10000))

Eager time: 0.43244256599996334
Graph time: 3.74799916500001


In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D

# Model building
inputs = Input(shape=(28, 28, 1)) 
x = Conv2D(filters = 3, kernel_size=(3, 3))(inputs)
x = Flatten()(x) 
x = Dense(256, "relu")(x)
x = Dense(256, "relu")(x) 
x = Dense(256, "relu")(x) 
outputs = Dense(10, "softmax")(x) 

input_data = tf.random.uniform([100, 28, 28])

# Eager Execution
eager_model = Model(inputs=inputs, outputs=outputs)
print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))

#Graph Execution 
graph_model = tf.function(eager_model) # Wrap the model with tf.function 
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))

Eager time: 30.05153006499995
Graph time: 6.977375514999949


## Differences of Two Modes

In [8]:
#!pip install -Uqq ipdb
import ipdb

### Debugging

In [11]:
@tf.function
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  ipdb.set_trace()
  x = x + b
  return x

In [12]:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)
orig_value = a_regular_function(x1, y1, b1).numpy()

> /tmp/__autograph_generated_file2yayz5wd.py(12)tf__a_regular_function()
     11                 ag__.ld(ipdb).set_trace()
---> 12                 x = (ag__.ld(x) + ag__.ld(b))
     13                 try:

ipdb> print(x)
Tensor("MatMul:0", shape=(1, 1), dtype=float32)
ipdb> print(x.numpy())
*** AttributeError: 'Tensor' object has no attribute 'numpy'
ipdb> exit


BdbQuit: ignored

### Print in Graph Mode

In [23]:
tf.config.run_functions_eagerly(False)
@tf.function
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  tf.print("matrix multipication applied")
  x = x + b
  return x

In [24]:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)
orig_value = a_regular_function(x1, y1, b1).numpy()

matrix multipication applied


In [25]:
x1 = tf.constant([[4.0, 8.0]])
y1 = tf.constant([[6.0], [7.0]])
b1 = tf.constant(1.0)
orig_value = a_regular_function(x1, y1, b1).numpy()

matrix multipication applied
